<a href="https://www.kaggle.com/code/samyakb/using-xgboost?scriptVersionId=158454249" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Read the data
X_full = pd.read_csv('/kaggle/input/playground-series-s4e1/train.csv', index_col='id')
X_test_full = pd.read_csv('/kaggle/input/playground-series-s4e1/test.csv', index_col='id')

# Remove rows with missing target, separate target from predictors
X_full.dropna(axis=0, subset=['Exited'], inplace=True)
y = X_full.Exited
X_full.drop(['CustomerId','Exited','CreditScore'], axis=1, inplace=True)

# Break off validation set from training data
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X_full, y, 
                                                                train_size=0.8, test_size=0.2,
                                                                random_state=0)

# Select categorical columns 
categorical_cols = [cname for cname in X_train_full.columns if
                    X_train_full[cname].dtype == "object"]

# Select numerical columns
numerical_cols = [cname for cname in X_train_full.columns if 
                X_train_full[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()
X_test = X_test_full[my_cols].copy()

In [2]:
X_train.head()

,Surname,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
id,,,,,,,,,,
15225,Hsu,France,Male,32.0,10,0.00,2,1.0,1.0,49463.44
28152,Omeokachie,France,Female,32.0,0,0.00,1,1.0,1.0,75578.67
117535,Hsieh,Spain,Male,43.0,10,123817.85,2,1.0,1.0,92910.53
92286,P'eng,France,Male,41.0,4,0.00,1,1.0,1.0,132798.58
133713,Padovano,France,Female,39.0,7,0.00,1,1.0,0.0,131763.66


In [3]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error


# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='constant') # Your code here

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Define the model
model = XGBRegressor(random_state=10, learning_rate=0.05) 


In [4]:
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])

# Preprocessing of training data, fit model 
my_pipeline.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
preds = my_pipeline.predict(X_valid)

# Evaluate the model
score = mean_absolute_error(y_valid, preds)
print('MAE:', score)

MAE: 0.20044457778000485


In [5]:
preds_test = my_pipeline.predict(X_test)

In [6]:
# Save test predictions to file
output = pd.DataFrame({'id': X_test.index,
                       'Exited': preds_test})
output.to_csv('submission.csv', index=False)